In [12]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import gmaps
from config import gkey
import requests
import json
from config import geokey



In [2]:
#defining ideal conditions
temp_max = 35
temp_min = 25
humidity_max=45
cloud_max=50
wind_max=7 #in meters/sec

In [3]:
filename = "Data_and_Output/weather.csv"
destination_df = pd.read_csv(filename, dtype = str)
destination_df = destination_df.astype({"Lat":"float64","Long":"float64", "Temp":"float64", "Humidity":"float64",
                                        "%Cloud":"float64","Wind_Speed":"float64"})
     

In [4]:
#creating heat map of humidity
gmaps.configure(gkey)

locations = destination_df[["Lat", "Long"]]
humidity = destination_df["Humidity"]


hum_max = humidity.max()
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '2px solid black',
    'padding': '4px'
}

fig = gmaps.figure(layout = figure_layout, map_type="TERRAIN", center = [0,0], zoom_level =1)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating = False, max_intensity = hum_max,  
                                 point_radius=4)
heat_layer.gradient = ["aqua","green", "yellow","red"]  #color codes from (https://www.w3.org/TR/css-color-3/#html4)
heat_layer.opacity = 0.3
fig.add_layer(heat_layer)

print("Heat Map Showing Humidity Variation at 500 Locations Chosen Randomly")

fig

Heat Map Showing Humidity Variation at 500 Locations Chosen Randomly


Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='4px', width='600px'))

In [5]:
#filtering data


destination_df = destination_df.loc[(destination_df["Temp"]<=temp_max)&(destination_df["Temp"]>=temp_min)]
destination_df = destination_df.loc[(destination_df["Humidity"]<=humidity_max)]
destination_df = destination_df.loc[(destination_df["%Cloud"]<=cloud_max)]
destination_df =destination_df.loc[(destination_df["%Cloud"]<=wind_max)]
destination_df

,City,Country,Lat,Long,Temp,Humidity,%Cloud,Wind_Speed,Hemisphere
36,bethel,us,44.8599,-168.7372,32.22,43.0,1.0,6.20,Northern
64,safaga,eg,24.9744,34.9866,35.00,26.0,0.0,7.20,Northern
152,canon city,us,38.5814,-105.6169,30.00,35.0,1.0,4.60,Northern
156,balkanabat,tm,41.0588,54.0674,34.51,29.0,0.0,4.49,Northern
163,biltine,td,15.4768,22.7041,33.87,32.0,0.0,3.33,Northern
176,twin falls,us,41.6476,-114.4561,26.67,29.0,1.0,3.60,Northern
185,irbil,iq,36.4230,44.1066,33.00,15.0,0.0,1.50,Northern
192,marshall,us,44.9385,-95.8771,32.22,43.0,1.0,2.60,Northern
208,camalu,mx,30.8405,-116.8769,26.67,41.0,5.0,2.68,Northern
253,hami,cn,45.7932,92.8392,29.22,20.0,0.0,8.13,Northern


In [6]:
destination_df.shape

(22, 9)

In [17]:
destination_df = destination_df.astype({"Lat":"object","Long":"object"})



loc_x = [destination_df.iloc[i][2] for i in range(0,len(destination_df))]
loc_y = [destination_df.iloc[i][3] for i in range(0,len(destination_df))]
loc = [(f"{loc_x[i]} , {loc_y[i]}") for i in range(0,len(loc_x))]


#target_search="Hotel"
target_search = "hotel"
target_radius = 10000
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


m=0 #m to track exceptions if needed
hotel = []
lat = []
long = []
for i in range(0,len(loc)):
            params = {
                   "location":loc[i],
                    "keyword":target_search,
                   "radius":target_radius,
                   "type":target_type,
                   "key":gkey
               }
            try:
                response = requests.get(base_url, params=params)
                data=response.json()
                
            except KeyError:
                m+=1
            if data["status"]!= "ZERO_RESULTS":
                hotel.append(data["results"][0]["name"])
                lat.append((data["results"][0]["geometry"]["location"]["lat"]))
                long.append((data["results"][0]["geometry"]["location"]["lng"]))
                
            else:
                hotel.append("No hotel in 5 km")
                lat.append((loc_x[i]))
                long.append((loc_y[i]))
                
         
            
            

In [18]:
print(hotel)

['No hotel in 5 km', 'Bedouin Valley Eco Lodge', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'Hotel Grand Swiss', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'هوتل شهرداری', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'No hotel in 5 km', 'Araguaia Hotel']


In [19]:
#assembling a dataframe with information
loc_vacation_df = pd.DataFrame(columns = ["Hotel", "Lat", "Long"])
for i in range(0,len(hotel)):
    if hotel[i]!= "No hotel in 5 km":        
        loc_x = lat[i]
        loc_y = long[i]
        loc_vacation_df = loc_vacation_df.append({"Hotel":hotel[i],"Lat":loc_x,"Long":loc_y},ignore_index = True)



In [20]:
loc_vacation_df

,Hotel,Lat,Long
0,Bedouin Valley Eco Lodge,24.965362,34.932579
1,Hotel Grand Swiss,36.374134,44.201255
2,هوتل شهرداری,36.911350,66.184928
3,Araguaia Hotel,-18.217484,-49.736272


In [21]:
loc_vacation_df["Hotel"].count()

4

In [22]:
print(response)

<Response [200]>


In [39]:
#https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY
country=[]
city = []
length_list = []
base_url="https://maps.googleapis.com/maps/api/geocode/json?"
for i in range(0,(loc_vacation_df["Hotel"].count())):
    lat = str(loc_vacation_df.iloc[i][1])
    long = str(loc_vacation_df.iloc[i][2])
    lat_long = f"{lat},{long}"
    query_url = base_url+"latlng="+lat_long+"&key="+gkey
    response = requests.get(query_url).json()
    length = len(response["results"])
    length_list.append(len(response["results"]))
    if length<9:
        city_p = length - 3
    else:
        city_p = length -4
    # print(response["results"][length-3]["address_components"][0]["long_name"])
    # print(response["results"][length-1]["formatted_address"])
    country.append(response["results"][length-1]["formatted_address"])
    city.append(response["results"][city_p]["address_components"][0]["long_name"])
    #print(json.dumps(response, indent=4, sort_keys=True))

In [46]:
loc_vacation_df["City"]=city
loc_vacation_df["Country"]=country
loc_vacation_df
loc_pair=[]
for i in range(0,len(loc_vacation_df)):
    loc_pair.append((loc_vacation_df.iloc[i][1],loc_vacation_df.iloc[i][2]))
    
print(loc_pair) 

[(24.965362, 34.9325791), (36.3741342, 44.201255), (36.9113504, 66.1849285), (-18.2174835, -49.73627219999999)]


In [47]:
#loc_vacation = [(-19,22 )]
fig1 = gmaps.figure(layout = figure_layout, map_type="TERRAIN", center = [0,0], zoom_level =1)
markers = gmaps.marker_layer(loc_pair)
fig1.add_layer(markers)
fig1

Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='4px', width='600px'))